In [1]:
import numpy as np
from model_functions import *
from Classification_models_LOSO_2Class import *

In [ ]:
#uncomment to run and create the all_psds file. 

#subject_data, freqs, targets = load_data(duration=30,overlap=15,seg_length=4)
#save_psds(subject_data,freqs,targets,'all_psds.pickle')


In [6]:
subject_data, freqs, targets = load_psds('all_psds.pickle')
test_labels = align_test_labels(classes=['A','C'])
features, targets = remove_class(subject_data,targets,'F')
features, targets = remove_test(features,targets,test_labels)
rbps = [relative_band_power(features[i],freqs,
                            freq_bands=[0.5,4.0,7.0,9.0,11.0,13.0,25.0, 45.0]) 
        for i in range(len(features))]

remove_channel function removes a list of channels before training and testing. This is done to see which channels contribute more to the discrimintating features between Alzheimer's and normal patients. 

In [8]:
rbps = remove_channel(rbps, ['Cz', 'P4'])

### Logistic Regression

In [ ]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 133)
train_metric_dict, val_metric_dict

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [ ]:
val_acc, train_acc

In [ ]:
PCA_components = [30, 33, 35, 37, 40]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [ ]:
val_acc, train_acc

In [ ]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 35)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 35,reg_parameter=100.0)
train_metric_dict, val_metric_dict

### k nearest neighbor classifier

In [ ]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 0)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 133)
train_metric_dict, val_metric_dict

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [ ]:
val_acc, train_acc

### Random Forest classifier

Here is a run of random Forest without doing PCA. As you can see in the definition of the method, PCA = 0, means we do not do PCA

In [ ]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 25, 
                                              min_samples_split = 4, PCA_components = 0)
train_metric_dict, val_metric_dict

Here is a run of random Forest with PCA. The maximum number of features is 133 here. 
As you can see, there is a significant reduction in validation set accuracy (from 75% to 71% )

In [ ]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 133)
train_metric_dict, val_metric_dict

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

### SVM

In [9]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='linear',
                                               reg_parameter=0.085, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9015024066545165,
  'sens': 0.9266101768377254,
  'spec': 0.8721357599107776,
  'f1': 0.9102569561237306},
 {'acc': 0.7831207065750736,
  'sens': 0.816747572815534,
  'spec': 0.7437899219304471,
  'f1': 0.8023845007451565})

In [ ]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',
                                               reg_parameter=1.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',
                                               reg_parameter=10.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',
                                               reg_parameter=100.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=1.0, PCA_components = 35)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=10.0, PCA_components = 35)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=1.0, PCA_components = 0)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='linear',
                                               reg_parameter=1.0,degree=2, PCA_components = 0)
train_metric_dict, val_metric_dict

### Multilayer Perceptron

In [ ]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 35, 
                                               max_iter = 600, hidden_layer_sizes=(3,1))
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 0, 
                                               max_iter = 500, hidden_layer_sizes=(10,1))
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 35, 
                                               alpha = 1.0, max_iter = 500, hidden_layer_sizes=(100,1))
train_metric_dict, val_metric_dict